# The feedforward neural network model for learning PageRank

In [1]:
import time
from helpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("fb_media_pages_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([4.26055693e-05])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 24000
target_pagerank[0]

array([1.02253366])

In [5]:
target_pagerank.shape

(27917, 1)

In [6]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("fb_media_pages_discounted_NDFC_matrix_r1_3_sta1_max70_rad5.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(27917, 6, 23)

In [8]:
matrix_1[111]

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, 2.00000e+00, 2.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00],
       [6.19050e-02, 0.00000e+00, 1.42860e-02, 4.17143e-01, 2.50476e-01,
        0.00000e+00, 1.56190e-01, 2.85710e-02, 2.85710e-02, 0.00000e+00,
        1.42860e-02, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 2.85710e-02, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.04390e-02, 1.12820e-02, 5.03200e-03, 4.17350e-02, 1.49669e-01,
        8.11320e-02, 2.63234e-01, 1.14913e-01, 1.24092e-01, 6.48310e-02,
        3.62280e-02, 2.18130e-02, 1.89210e-02, 1.55690e-02, 5.23000e-03,
        1.06400e-03, 1.82120e-02, 2.08300e-03, 0.00000e+00, 2.43800e-03,
        0.00000e+00, 2.08300

In [9]:
matrix_1 = matrix_1.reshape(27917,-1)
matrix_1.shape

(27917, 138)

# Converting the data into pytorch tenors

In [10]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([27917, 138]), torch.Size([27917, 1]))

In [11]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[1.0225],
        [0.9398],
        [0.8766],
        ...,
        [0.4565],
        [0.9406],
        [0.9156]], dtype=torch.float64, requires_grad=True)

In [12]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 7917
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([20000, 138]),
 torch.Size([7917, 138]),
 torch.Size([20000, 1]),
 torch.Size([7917, 1]))

## A function for dividing train data into batches

In [13]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [14]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [15]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=138, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [16]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

55200
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
549993


In [17]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [18]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [19]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.71976703405,    TEST LOSS:   0.65250766277
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.16982179880,    TEST LOSS:   0.53423196077
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.11732210964,    TEST LOSS:   0.47369492054
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.08380837739,    TEST LOSS:   0.43664672971
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.06691044569,    TEST LOSS:   0.40498298407
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.04758772999,    TEST LOSS:   0.38414224982
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.03564901650,    TEST LOSS:   0.36788755655
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.02899507247,    TEST LOSS:   0.35632988811
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [20]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,138)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%50 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 0.27503449374617245,    prediction: 0.29820671677589417
index 0:       inaccuracy: 8.425%
-----------------------------------------------------------------
target 1.5128024376814098,    prediction: 1.1485130786895752
index 50:       inaccuracy: 24.08%
-----------------------------------------------------------------
target 1.3425328665875027,    prediction: 1.362220287322998
index 100:       inaccuracy: 1.466%
-----------------------------------------------------------------
target 0.8115875830747279,    prediction: 0.6854863166809082
index 150:       inaccuracy: 15.538%
-----------------------------------------------------------------
target 0.5169964031324928,    prediction: 0.5144546031951904
index 200:       inaccuracy: 0.492%
-----------------------------------------------------------------
target 0.7102283365242272,    prediction: 0.6734765768051147
index 250:       inaccuracy: 5.175%
-----------------------------------------------------------------
target 1.83244782257461

target 2.1778671079396403,    prediction: 2.067462205886841
index 2550:       inaccuracy: 5.069%
-----------------------------------------------------------------
target 1.296581726686667,    prediction: 1.299121379852295
index 2600:       inaccuracy: 0.196%
-----------------------------------------------------------------
target 0.42147220243767586,    prediction: 0.4567713141441345
index 2650:       inaccuracy: 8.375%
-----------------------------------------------------------------
target 0.576293145575791,    prediction: 0.5671300292015076
index 2700:       inaccuracy: 1.59%
-----------------------------------------------------------------
target 1.6463739970014888,    prediction: 1.762951135635376
index 2750:       inaccuracy: 7.081%
-----------------------------------------------------------------
target 0.4704349001123442,    prediction: 0.5390045046806335
index 2800:       inaccuracy: 14.576%
-----------------------------------------------------------------
target 0.59189397608

target 0.2748986200768778,    prediction: 0.31466081738471985
index 5200:       inaccuracy: 14.464%
-----------------------------------------------------------------
target 0.38752940756884907,    prediction: 0.415834903717041
index 5250:       inaccuracy: 7.304%
-----------------------------------------------------------------
target 0.22421401900955504,    prediction: 0.28590598702430725
index 5300:       inaccuracy: 27.515%
-----------------------------------------------------------------
target 1.067787149602846,    prediction: 1.0941102504730225
index 5350:       inaccuracy: 2.465%
-----------------------------------------------------------------
target 0.29290119075511145,    prediction: 0.3252904713153839
index 5400:       inaccuracy: 11.058%
-----------------------------------------------------------------
target 0.49095924038446004,    prediction: 0.4295433759689331
index 5450:       inaccuracy: 12.509%
-----------------------------------------------------------------
target 0

target 0.6183388672832796,    prediction: 0.6610074639320374
index 7750:       inaccuracy: 6.901%
-----------------------------------------------------------------
target 0.2997575884002859,    prediction: 0.35303613543510437
index 7800:       inaccuracy: 17.774%
-----------------------------------------------------------------
target 0.18546377392151883,    prediction: 0.2689989507198334
index 7850:       inaccuracy: 45.041%
-----------------------------------------------------------------
target 0.8980913284581304,    prediction: 0.7655067443847656
index 7900:       inaccuracy: 14.763%
-----------------------------------------------------------------


In [21]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  13.143


In [22]:
torch.save(model, "trained_FFNN_model_fb_media_pagerank_discounted.pt")